# Overview

With this mini project, you will exercise using Spark transformations to solve traditional
MapReduce data problems. It demonstrates Spark having a significant advantage against
Hadoop MapReduce framework, in both code simplicity and its in-memory processing
performance, which best suit for the chain of MapReduce use cases.

Consider an automobile tracking platform that keeps track of history of incidents after a new
vehicle is sold by the dealer. Such incidents include further private sales, repairs and accident
reports. This provides a good reference for second hand buyers to understand the vehicles they
are interested in.

# Objective:
Find total number of accidents per make and year of the car.

## Data:

In [109]:
s = [('incident_id', 'INT',''), \
     ('incident_type','STRING','(I: initial sale, A: accident, R: repair)'), \
     ('vin_number','STRING',''), \
     ('make','STRING','(The brand of the car, only populated with incident type “I”)'), \
     ('model','STRING','(The model of the car, only populated with incident type “I”)'), \
     ('year','STRING','(The year of the car, only populated with incident type “I”)') ,\
     ('incident_date','DATE','(Date of the incident occurrence)'), \
     ('description','STRING','')]
df_schema = spark.createDataFrame(s, "Column: string, Type: string, Info: string")
df_schema.show(truncate=False)

+-------------+------+-------------------------------------------------------------+
|Column       |Type  |Info                                                         |
+-------------+------+-------------------------------------------------------------+
|incident_id  |INT   |                                                             |
|incident_type|STRING|(I: initial sale, A: accident, R: repair)                    |
|vin_number   |STRING|                                                             |
|make         |STRING|(The brand of the car, only populated with incident type “I”)|
|model        |STRING|(The model of the car, only populated with incident type “I”)|
|year         |STRING|(The year of the car, only populated with incident type “I”) |
|incident_date|DATE  |(Date of the incident occurrence)                            |
|description  |STRING|                                                             |
+-------------+------+-------------------------------------------

## Step 1. Filter out accident incidents with make and year
Since we only care about accident records, we should filter out records having incident type
other than “I”. However, accident records don't carry make and year fields due to the design to
remove redundancy. So our first step is propagating make and year info from record type I into
all other record types.

### 1.1 Read the input data CSV file
Use the Spark context object to read the input file to create the input RDD:

`sc = SparkContext("local", "My Application")`
<br>
`raw_rdd = sc.textFile("data.csv")`

Because pyspark kernel for jupyter notebook automatically initializes SparkContext, etc - it's not required
But in python script used by run.sh, the following commands are required to initialize the environment:

`import pyspark`
<br>
`from pyspark.sql import SparkSession`
<br>
`from pyspark import SparkContext`

In [138]:
#pyspark kernel for jupyter notebook automatically initializes SparkContext variable sc
spark.createDataFrame(sc.getConf().getAll(), "Configuration: string, Value: string").show(truncate = False)
raw_rdd = sc.textFile("data.csv")
df_schema.show(truncate=False)
raw_rdd.collect()

+----------------------------------+--------------------------------------------------+
|Configuration                     |Value                                             |
+----------------------------------+--------------------------------------------------+
|spark.app.startTime               |1654767472643                                     |
|spark.executor.id                 |driver                                            |
|spark.app.name                    |PySparkShell                                      |
|spark.app.id                      |local-1654767473552                               |
|spark.driver.port                 |38551                                             |
|spark.sql.catalogImplementation   |hive                                              |
|spark.rdd.compress                |True                                              |
|spark.serializer.objectStreamReset|100                                               |
|spark.master                   

['1,I,VXIO456XLBB630221,Nissan,Altima,2003,2002-05-08,Initial sales from TechMotors',
 '2,I,INU45KIOOPA343980,Mercedes,C300,2015,2014-01-01,Sold from EuroMotors',
 '3,A,VXIO456XLBB630221,,,,2014-07-02,Head on collision',
 '4,R,VXIO456XLBB630221,,,,2014-08-05,Repair transmission',
 '5,I,VOME254OOXW344325,Mercedes,E350,2015,2014-02-01,Sold from Carmax',
 '6,R,VOME254OOXW344325,,,,2015-02-06,Wheel allignment service',
 '7,R,VXIO456XLBB630221,,,,2015-01-01,Replace right head light',
 '8,I,EXOA00341AB123456,Mercedes,SL550,2016,2015-01-01,Sold from AceCars',
 '9,A,VOME254OOXW344325,,,,2015-10-01,Side collision',
 '10,R,VOME254OOXW344325,,,,2015-09-01,Changed tires',
 '11,R,EXOA00341AB123456,,,,2015-05-01,Repair engine',
 '12,A,EXOA00341AB123456,,,,2015-05-03,Vehicle rollover',
 '13,R,VOME254OOXW344325,,,,2015-09-01,Replace passenger side door',
 '14,I,UXIA769ABCC447906,Toyota,Camery,2017,2016-05-08,Initial sales from Carmax',
 '15,R,UXIA769ABCC447906,,,,2020-01-02,Initial sales from Carmax',

### 1.1 Perform map operation
We need to propagate make and year to the accident records (incident type A), using
vin_number as the aggregate key. Therefore the map output key should be vin_number, value
should be the make and year, along with the incident type. In Spark, in order to proceed with the
“groupByKey” function, we need the map operation to produce PairRDD, with tuple type as each
record.

`vin_kv = raw_rdd.map(lambda x: extract_vin_key_value(x))`
<br>
`# Please implement method extract_vin_key_value()`

In [308]:
vin_kv = raw_rdd.map(lambda x: (x.split(',')[2],[x.split(',')[3],x.split(',')[5],x.split(',')[1]]))
vin_kv.collect()

[('VXIO456XLBB630221', ['Nissan', '2003', 'I']),
 ('INU45KIOOPA343980', ['Mercedes', '2015', 'I']),
 ('VXIO456XLBB630221', ['', '', 'A']),
 ('VXIO456XLBB630221', ['', '', 'R']),
 ('VOME254OOXW344325', ['Mercedes', '2015', 'I']),
 ('VOME254OOXW344325', ['', '', 'R']),
 ('VXIO456XLBB630221', ['', '', 'R']),
 ('EXOA00341AB123456', ['Mercedes', '2016', 'I']),
 ('VOME254OOXW344325', ['', '', 'A']),
 ('VOME254OOXW344325', ['', '', 'R']),
 ('EXOA00341AB123456', ['', '', 'R']),
 ('EXOA00341AB123456', ['', '', 'A']),
 ('VOME254OOXW344325', ['', '', 'R']),
 ('UXIA769ABCC447906', ['Toyota', '2017', 'I']),
 ('UXIA769ABCC447906', ['', '', 'R']),
 ('INU45KIOOPA343980', ['', '', 'A'])]

### 1.2 Perform group aggregation to populate make and year to all the records
Like the reducer in MapReduce framework, Spark provides a “groupByKey” function to achieve
shuffle and sort in order to aggregate all records sharing the same key to the same groups.
Within a group of vin_number, we need to iterate through all the records and find the one that
has the make and year available and capture it in group level master info. As we filter and
output accident records, those records need to be modified adding the master info that we
captured in the first iteration.

`enhance_make = vin_kv.groupByKey().flatMap(lambda kv: populate_make(kv[1]))`
<br>
`# Please implement method populate_make()`

In [313]:
enhance_make = vin_kv.groupByKey(). \
    flatMap(lambda x: [(item[0],item[1]) for (item) in x[1]]). \
    filter(lambda x:x[0]!='' and x[1]!='')

In [314]:
enhance_make.collect()

[('Mercedes', '2015'),
 ('Mercedes', '2016'),
 ('Toyota', '2017'),
 ('Nissan', '2003'),
 ('Mercedes', '2015')]

## Step 2. Count number of occurrence for accidents for the vehicle make and year

### 2.1 Perform map operation
The goal of this step is to count the number of records for each make and year combination,
given the result we derived previously. The output key should be the combination of vehicle
make and year. The value should be the count of 1.

`make_kv = enhance_make.map(lambda x: extract_make_key_value(x))`
<br>
`# Please implement method extract_make_key_value()`

In [324]:
make_kv = enhance_make.map(lambda x: ((x[0],x[1]),1))

In [325]:
make_kv.collect()

[(('Mercedes', '2015'), 1),
 (('Mercedes', '2016'), 1),
 (('Toyota', '2017'), 1),
 (('Nissan', '2003'), 1),
 (('Mercedes', '2015'), 1)]

### 2.2 Aggregate the key and count the number of records in total per key
Use Spark provided “reduceByKey” function to perform the sum of all the values (1) from each
record. As a result, we get the make and year combination key along with its total record count.

In [327]:
from operator import add
acc_make_year=make_kv.reduceByKey(add)
acc_make_year.collect()

[(('Mercedes', '2015'), 2),
 (('Mercedes', '2016'), 1),
 (('Toyota', '2017'), 1),
 (('Nissan', '2003'), 1)]

## Step 3. Save the result to HDFS as text
The output file should look similar to this.

`Nissan-2003,1`
<br>
`BMW-2008,10`
<br>
`MERCEDES-2013,2`

In [347]:
# First to plain text file for sanity check
file = open('output.txt', 'x')
for acc in acc_make_year.collect():
    file.write('{}-{},{}\n'.format(acc[0][0], acc[0][1], acc[1]))
file.close()

# Now in HDFS file format
out = []
for acc in acc_make_year.collect():
    out.append('{}-{},{}'.format(acc[0][0], acc[0][1], acc[1]))
rdd_out = sc.parallelize(out)
rdd_out.saveAsTextFile('HDFS_text_results')

## Step 4. Shell script to run the Spark jobs
To run your Spark code, we need to use command “spark-submit” to start a Spark application
and specify your Python script. Suppose your script is named autoinc_spark.py, your shell script
should look like this.

`spark-submit autoinc_spark.py`

My run.sh file consists of:
    
`#!/bin/sh`
<br>
`python autoinc_spark.py > execution_log.txt`
<br>
`echo '\nsingle file results printed to output.txt'`
<br>
`echo '\nresults saved as text file in HDFS format in HDFS_text_results folder'`
<br>

Running `./run.sh` produces file output.txt and partitions in HDFS_text folder containing the following records:

`Nissan-2003,1`
<br>
`Mercedes-2015,2`
<br>
`Mercedes-2016,1`
<br>
`Toyota-2017,1`